# Combine TR x contrast for CNR

In [1]:
from nipype.interfaces.io import BIDSDataGrabber, DataFinder, DataSink, DataGrabber
import nipype.pipeline as pe
import nipype as ni
import nipype.interfaces.fsl.maths as fsl
from nipype.algorithms.misc import Gunzip 
import pandas as pd
import os, re, json

In [2]:
Basedir = "/scratch/qbi/uqkgarn1/STRIWP1/"
CNRcomb = pe.Workflow(name="CNRcomb") # workflow to run the analysis

In [3]:
# THIS CODE DEFINITELY WORKS
dg = pe.Node(DataGrabber(infields=['sub', 'TR'], 
                         outfields=['CN']), name='data-grabber')
dg.inputs.base_dir = "/scratch/qbi/uqkgarn1/STRIWP1/"
dg.inputs.sort_filelist = True
dg.inputs.template='*'
dg.inputs.template_args = {'CN': [['sub', 'TR']]} 
dg.inputs.field_template = {'CN': '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-%s/TR%s/MASKS/CNR/CN_*.nii.gz'} 


In [4]:
def printSubPath(fullFilePath):
    # function to split filepath into constituent parts, then print string to add as input to DataSink for the container string
    # given the full filepath, this extracts the subject folder and TR strings for input
    # into DataSink
    import os
    import re
    fname = os.path.normpath(fullFilePath[0])
    l = fname.split(os.sep)
    name = [s for s in l if re.search('sub', s)][0], [t for t in l if re.search('TR([0-9])', t)][0]
    name = '/'.join(name)
    name
    return name

In [5]:
ds = pe.Node(DataSink(), name='sink-stuff')
ds.inputs.base_directory = "/scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/"
substitutions = [('_TR_([0-9]*)_sub_([0-9]*)', ''),
                 ('_gunzipmask([0-9]*)', ''),
                 ('_mask-CNR([0-9]*)', ''),
                 ('_mask-RMS([0-9]*)', '')]
ds.inputs.regexp_substitutions = substitutions

In [6]:
# THIS WORKS
datinf = pe.Node(ni.IdentityInterface(fields=['sub', 'TR']),
                 name='dat-info')
datinf.iterables = [('sub', ['01', '02', '03', '04', '05']), ('TR', ['700', '1510', '1920'])]
#datinf.iterables = [('sub', ['05']), ('TR', ['1920'])]
datinf.iterables 

[('sub', ['01', '02', '03', '04', '05']), ('TR', ['700', '1510', '1920'])]

# Add images together

Make a function that uses one output - e.g. CN, to create a list of the rest, so its formatted for multiimage maths

In [7]:
def getimages(in_files):    
    masks = ['FEF_', 'IPS_', 'LOC_', 'GPe_', 'GPi_', 'Put_', 'STN_', 'VS_']
    out_files = [ [in_files[i].replace('CN_', m) for m in masks] for i in range(len(in_files)) ]
    return out_files

# Define Image Maths

In [8]:
domaths = pe.MapNode(fsl.MultiImageMaths(), name='do-maths', iterfield=['in_file', 'operand_files', 'out_file'])
domaths.inputs.op_string = '-add %s -add %s -add %s -add %s -add %s -add %s -add %s -add %s'

In [9]:
def getoutputname(in_files):
    return( [in_files[i].replace('CN_', 'ALLROIS_') for i in range(len(in_files))] )

In [10]:
CNRcomb.connect([(datinf, dg, [('sub', 'sub')]),
                 (datinf, dg, [('TR', 'TR')]),
                 (dg, ds, [(('CN', printSubPath), 'container')]),
                 (dg, domaths, [('CN', 'in_file')]),
                 (dg, domaths, [(('CN', getimages), 'operand_files')]),
                 (dg, domaths, [(('CN', getoutputname), 'out_file')]),
                 (domaths, ds, [('out_file', 'MASKS.CNR')])
                ])
CNRcomb.run()

200914-20:17:46,765 nipype.workflow INFO:
	 Workflow CNRcomb settings: ['check', 'execution', 'logging', 'monitoring']
200914-20:17:46,836 nipype.workflow INFO:
	 Running serially.
200914-20:17:46,837 nipype.workflow INFO:
	 [Node] Setting-up "CNRcomb.data-grabber" in "/tmp/tmpc3hdt4d8/CNRcomb/_TR_1920_sub_05/data-grabber".
200914-20:17:46,843 nipype.workflow INFO:
	 [Node] Running "data-grabber" ("nipype.interfaces.io.DataGrabber")
200914-20:17:46,850 nipype.workflow INFO:
	 [Node] Finished "CNRcomb.data-grabber".
200914-20:17:46,852 nipype.workflow INFO:
	 [Node] Setting-up "CNRcomb.do-maths" in "/tmp/tmp1zxkvqyg/CNRcomb/_TR_1920_sub_05/do-maths".
200914-20:17:46,869 nipype.workflow INFO:
	 [Node] Setting-up "_do-maths0" in "/tmp/tmp1zxkvqyg/CNRcomb/_TR_1920_sub_05/do-maths/mapflow/_do-maths0".
200914-20:17:46,876 nipype.workflow INFO:
	 [Node] Running "_do-maths0" ("nipype.interfaces.fsl.maths.MultiImageMaths"), a CommandLine Interface with command:
fslmaths /scratch/qbi/uqkgarn1/ST

200914-20:17:48,824 nipype.workflow INFO:
	 [Node] Finished "_do-maths5".
200914-20:17:48,826 nipype.workflow INFO:
	 [Node] Setting-up "_do-maths6" in "/tmp/tmp1zxkvqyg/CNRcomb/_TR_1920_sub_05/do-maths/mapflow/_do-maths6".
200914-20:17:48,832 nipype.workflow INFO:
	 [Node] Running "_do-maths6" ("nipype.interfaces.fsl.maths.MultiImageMaths"), a CommandLine Interface with command:
fslmaths /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-05/TR1920/MASKS/CNR/CN_CNR_0009.nii.gz -add /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-05/TR1920/MASKS/CNR/FEF_CNR_0009.nii.gz -add /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-05/TR1920/MASKS/CNR/IPS_CNR_0009.nii.gz -add /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-05/TR1920/MASKS/CNR/LOC_CNR_0009.nii.gz -add /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-05/TR1920/MASKS/CNR/GPe_CNR_0009.nii.gz -add /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-05/TR1920/MASKS/CNR/GPi_CNR_0009.nii.gz -add /scratch/qbi/uqkgarn1/STRIWP1/derivativ

200914-20:17:50,813 nipype.workflow INFO:
	 [Node] Finished "_do-maths4".
200914-20:17:50,816 nipype.workflow INFO:
	 [Node] Setting-up "_do-maths5" in "/tmp/tmpf3bjgbys/CNRcomb/_TR_1510_sub_05/do-maths/mapflow/_do-maths5".
200914-20:17:50,821 nipype.workflow INFO:
	 [Node] Running "_do-maths5" ("nipype.interfaces.fsl.maths.MultiImageMaths"), a CommandLine Interface with command:
fslmaths /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-05/TR1510/MASKS/CNR/CN_CNR_0007.nii.gz -add /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-05/TR1510/MASKS/CNR/FEF_CNR_0007.nii.gz -add /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-05/TR1510/MASKS/CNR/IPS_CNR_0007.nii.gz -add /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-05/TR1510/MASKS/CNR/LOC_CNR_0007.nii.gz -add /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-05/TR1510/MASKS/CNR/GPe_CNR_0007.nii.gz -add /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-05/TR1510/MASKS/CNR/GPi_CNR_0007.nii.gz -add /scratch/qbi/uqkgarn1/STRIWP1/derivativ

200914-20:17:52,265 nipype.workflow INFO:
	 [Node] Finished "_do-maths3".
200914-20:17:52,267 nipype.workflow INFO:
	 [Node] Setting-up "_do-maths4" in "/tmp/tmp_jt6h60s/CNRcomb/_TR_700_sub_05/do-maths/mapflow/_do-maths4".
200914-20:17:52,272 nipype.workflow INFO:
	 [Node] Running "_do-maths4" ("nipype.interfaces.fsl.maths.MultiImageMaths"), a CommandLine Interface with command:
fslmaths /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-05/TR700/MASKS/CNR/CN_CNR_0006.nii.gz -add /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-05/TR700/MASKS/CNR/FEF_CNR_0006.nii.gz -add /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-05/TR700/MASKS/CNR/IPS_CNR_0006.nii.gz -add /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-05/TR700/MASKS/CNR/LOC_CNR_0006.nii.gz -add /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-05/TR700/MASKS/CNR/GPe_CNR_0006.nii.gz -add /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-05/TR700/MASKS/CNR/GPi_CNR_0006.nii.gz -add /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/

200914-20:17:53,888 nipype.workflow INFO:
	 [Node] Finished "_do-maths2".
200914-20:17:53,891 nipype.workflow INFO:
	 [Node] Setting-up "_do-maths3" in "/tmp/tmps_zm0y8j/CNRcomb/_TR_1920_sub_04/do-maths/mapflow/_do-maths3".
200914-20:17:53,897 nipype.workflow INFO:
	 [Node] Running "_do-maths3" ("nipype.interfaces.fsl.maths.MultiImageMaths"), a CommandLine Interface with command:
fslmaths /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-04/TR1920/MASKS/CNR/CN_CNR_0005.nii.gz -add /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-04/TR1920/MASKS/CNR/FEF_CNR_0005.nii.gz -add /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-04/TR1920/MASKS/CNR/IPS_CNR_0005.nii.gz -add /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-04/TR1920/MASKS/CNR/LOC_CNR_0005.nii.gz -add /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-04/TR1920/MASKS/CNR/GPe_CNR_0005.nii.gz -add /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-04/TR1920/MASKS/CNR/GPi_CNR_0005.nii.gz -add /scratch/qbi/uqkgarn1/STRIWP1/derivativ

...

200914-20:18:10,638 nipype.workflow INFO:
	 [Node] Finished "_do-maths0".
200914-20:18:10,640 nipype.workflow INFO:
	 [Node] Setting-up "_do-maths1" in "/tmp/tmpv_k60hwj/CNRcomb/_TR_700_sub_02/do-maths/mapflow/_do-maths1".
200914-20:18:10,649 nipype.workflow INFO:
	 [Node] Running "_do-maths1" ("nipype.interfaces.fsl.maths.MultiImageMaths"), a CommandLine Interface with command:
fslmaths /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-02/TR700/MASKS/CNR/CN_CNR_0002.nii.gz -add /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-02/TR700/MASKS/CNR/FEF_CNR_0002.nii.gz -add /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-02/TR700/MASKS/CNR/IPS_CNR_0002.nii.gz -add /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-02/TR700/MASKS/CNR/LOC_CNR_0002.nii.gz -add /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-02/TR700/MASKS/CNR/GPe_CNR_0002.nii.gz -add /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-02/TR700/MASKS/CNR/GPi_CNR_0002.nii.gz -add /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/

200914-20:18:11,870 nipype.workflow INFO:
	 [Node] Running "sink-stuff" ("nipype.interfaces.io.DataSink")
200914-20:18:11,876 nipype.workflow INFO:
	 [Node] Finished "CNRcomb.sink-stuff".
200914-20:18:11,876 nipype.workflow INFO:
	 [Node] Setting-up "CNRcomb.data-grabber" in "/tmp/tmp3n6wbya4/CNRcomb/_TR_1920_sub_01/data-grabber".
200914-20:18:11,881 nipype.workflow INFO:
	 [Node] Running "data-grabber" ("nipype.interfaces.io.DataGrabber")
200914-20:18:11,885 nipype.workflow INFO:
	 [Node] Finished "CNRcomb.data-grabber".
200914-20:18:11,886 nipype.workflow INFO:
	 [Node] Setting-up "CNRcomb.do-maths" in "/tmp/tmpbctj_xtj/CNRcomb/_TR_1920_sub_01/do-maths".
200914-20:18:11,901 nipype.workflow INFO:
	 [Node] Setting-up "_do-maths0" in "/tmp/tmpbctj_xtj/CNRcomb/_TR_1920_sub_01/do-maths/mapflow/_do-maths0".
200914-20:18:11,906 nipype.workflow INFO:
	 [Node] Running "_do-maths0" ("nipype.interfaces.fsl.maths.MultiImageMaths"), a CommandLine Interface with command:
fslmaths /scratch/qbi/uqkg

200914-20:18:14,1 nipype.workflow INFO:
	 [Node] Finished "_do-maths5".
200914-20:18:14,4 nipype.workflow INFO:
	 [Node] Setting-up "_do-maths6" in "/tmp/tmpbctj_xtj/CNRcomb/_TR_1920_sub_01/do-maths/mapflow/_do-maths6".
200914-20:18:14,11 nipype.workflow INFO:
	 [Node] Running "_do-maths6" ("nipype.interfaces.fsl.maths.MultiImageMaths"), a CommandLine Interface with command:
fslmaths /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-01/TR1920/MASKS/CNR/CN_CNR_0009.nii.gz -add /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-01/TR1920/MASKS/CNR/FEF_CNR_0009.nii.gz -add /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-01/TR1920/MASKS/CNR/IPS_CNR_0009.nii.gz -add /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-01/TR1920/MASKS/CNR/LOC_CNR_0009.nii.gz -add /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-01/TR1920/MASKS/CNR/GPe_CNR_0009.nii.gz -add /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-01/TR1920/MASKS/CNR/GPi_CNR_0009.nii.gz -add /scratch/qbi/uqkgarn1/STRIWP1/derivatives/gl

200914-20:18:16,145 nipype.workflow INFO:
	 [Node] Finished "_do-maths4".
200914-20:18:16,147 nipype.workflow INFO:
	 [Node] Setting-up "_do-maths5" in "/tmp/tmp75hp9zc8/CNRcomb/_TR_1510_sub_01/do-maths/mapflow/_do-maths5".
200914-20:18:16,154 nipype.workflow INFO:
	 [Node] Running "_do-maths5" ("nipype.interfaces.fsl.maths.MultiImageMaths"), a CommandLine Interface with command:
fslmaths /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-01/TR1510/MASKS/CNR/CN_CNR_0007.nii.gz -add /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-01/TR1510/MASKS/CNR/FEF_CNR_0007.nii.gz -add /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-01/TR1510/MASKS/CNR/IPS_CNR_0007.nii.gz -add /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-01/TR1510/MASKS/CNR/LOC_CNR_0007.nii.gz -add /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-01/TR1510/MASKS/CNR/GPe_CNR_0007.nii.gz -add /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-01/TR1510/MASKS/CNR/GPi_CNR_0007.nii.gz -add /scratch/qbi/uqkgarn1/STRIWP1/derivativ

200914-20:18:17,715 nipype.workflow INFO:
	 [Node] Finished "_do-maths3".
200914-20:18:17,718 nipype.workflow INFO:
	 [Node] Setting-up "_do-maths4" in "/tmp/tmpj60w3p2b/CNRcomb/_TR_700_sub_01/do-maths/mapflow/_do-maths4".
200914-20:18:17,723 nipype.workflow INFO:
	 [Node] Running "_do-maths4" ("nipype.interfaces.fsl.maths.MultiImageMaths"), a CommandLine Interface with command:
fslmaths /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-01/TR700/MASKS/CNR/CN_CNR_0006.nii.gz -add /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-01/TR700/MASKS/CNR/FEF_CNR_0006.nii.gz -add /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-01/TR700/MASKS/CNR/IPS_CNR_0006.nii.gz -add /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-01/TR700/MASKS/CNR/LOC_CNR_0006.nii.gz -add /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-01/TR700/MASKS/CNR/GPe_CNR_0006.nii.gz -add /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-01/TR700/MASKS/CNR/GPi_CNR_0006.nii.gz -add /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/